In [17]:
import json
import os

timestamps = [i * 0.02 for i in range(1500 + 1)]
replace = {
    'Aee.': 'A.',
    'Bee.': 'B.',
    'See.': 'C.',
    'Dee.': 'D.',
}

In [18]:
with open('tatabahasa.json') as fopen:
    d = json.load(fopen)
    
len(d)

1284

In [21]:
result = []

for i in range(len(d)):
    audio = os.path.join('tatabahasa-v2', f'{i}.mp3')
    align = os.path.join('tatabahasa-v2-alignment_alignment', f'{i}.json')
    
    if not os.path.exists(audio):
        continue
    
    with open(align) as fopen:
        align = json.load(fopen)
    
    merged_data = []
    for entry in align[1:]:
        if entry['text'] == '.' and merged_data:
            merged_data[-1]['text'] += '.'
            merged_data[-1]['end'] = entry['end']
        else:
            merged_data.append(entry)

    for i in range(len(merged_data)):
        merged_data[i]['text'] = replace.get(merged_data[i]['text'], merged_data[i]['text'])
        
    ts = []
    for c_ in merged_data:
        start = min(timestamps, key=lambda t: abs(t - (c_['start'])))
        end = min(timestamps, key=lambda t: abs(t - (c_['end'])))
        w = c_['text']
        t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
        ts.append(t)

    ts = ''.join(ts)
    new_text = text = f"<|startoftranscript|><|ms|><|transcribeprecise|>{ts}<|endoftext|>"

    result.append({
        'new_text': new_text,
        'audio_filename': audio,
    })

In [22]:
len(result)

671

In [27]:
!zip -rq tatabahasa-v2.zip tatabahasa-v2

In [25]:
import pandas as pd

pd.DataFrame(result).to_parquet('tatabahasa-word-timestamps.parquet')

In [26]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="tatabahasa-word-timestamps.parquet",
    path_in_repo="data/tatabahasa_word-00000-of-00001.parquet",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

tatabahasa-word-timestamps.parquet:   0%|          | 0.00/255k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/1a3936a87d979aacaa55fdef0b4a4d1f42541c23', commit_message='Upload data/tatabahasa_word-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='1a3936a87d979aacaa55fdef0b4a4d1f42541c23', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)

In [28]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="tatabahasa-v2.zip",
    path_in_repo="tatabahasa-v2.zip",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

tatabahasa-v2.zip:   0%|          | 0.00/65.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/978ea8fec4a813ec085fea2a3e0cbb8c5af1e60c', commit_message='Upload tatabahasa-v2.zip with huggingface_hub', commit_description='', oid='978ea8fec4a813ec085fea2a3e0cbb8c5af1e60c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)